In [6]:
import os, sys
import numpy as np
import math

In [7]:
def point_cloud_center(x_coords, y_coords, weights):
    """
    Find weighted center of gravity of point cloud
    Use the column "weight_col" values as weight factor (should be numeric)
    Return (x,y)
    """
    sum_weights = sum(weights)
    # Catch cases where all weights are zero (or missing)
    if (sum_weights==0):
        # Just use regular centroid
        n = len(x_coords)
        x = sum(x_coords)/n
        y = sum(y_coords)/n
    else:
        # Do the weighted center of gravity
        # Since weights and coords are numpy arrays, '*' is array multiplication
        x = sum(x_coords * weights) / sum_weights
        y = sum(y_coords * weights) / sum_weights
    
    # Calculate Standard distance
    sigma_x = np.sum(x_coords**2 * weights)/sum_weights - x**2
    sigma_y = np.sum(y_coords**2 * weights)/sum_weights - y**2
    d = np.sqrt(sigma_x + sigma_y)
    
    return x,y,d

In [8]:
# Initialize GRASS Session

# --- Change variable below as required ---
# -----------------------------------------
gisdb = os.path.join(os.path.expanduser("~"), "GIS/grass")
gisbase = '/usr/lib/grass72'
grass7bin = '/usr/bin/grass72'
location="WGS84"
mapset="IMAP"
# -----------------------------------------

gpydir = os.path.join(gisbase, "etc", "python")
sys.path.append(gpydir)
os.environ['GISBASE'] = gisbase
os.environ['GISDBASE'] = gisdb

import grass.script as gscript
import grass.script.setup as gsetup
tmprc = gsetup.init(gisbase, gisdb, location, mapset)
# (Remove the tmprc file at the end)

In [9]:
def dispersion_cross(cog, x_coords, y_coords, x, y):
    """
    Create line vector of dispersion cross
    No weighting
    """
    x_std = np.std(x_coords)
    y_std = np.std(y_coords)
    r = np.corrcoef(x_coords, y_coords)[0,1]
    tan2alpha = 2*r*x_std*y_std/(x_std - y_std)
    alpha = math.atan(tan2alpha)/2
    alpha_deg = math.degrees(alpha)
    
    # Calculate new, transformed X,Y corrds
    x_star = x_coords*math.cos(alpha) + y_coords*math.sin(alpha)
    y_star = y_coords*math.cos(alpha) - x_coords*math.sin(alpha)
    
    # Now the lengths of dispersion cross segments = Standard deviation of each axis
    x_len = np.std(x_star)
    y_len = np.std(y_star)
    disp_ellipse = cog.replace('cog','disp_ellipse')
    gscript.run_command('v.buffer', 
        input=cog, output=disp_ellipse, type='point',
        distance=x_len, minordistance=y_len, angle=alpha_deg, overwrite=True)
    
    return alpha_deg, x_len, y_len